# Multi-Region IoT with ACM PCA

These Jupyter notebooks contain some code snippets for the re:Invent 2019 chalk talk (IOT335) about multi-region IoT setups. The setup assumes that you have a master and a slave region where you want to connect your devices to.

AWS infrastructure like [AWS IoT topic rules](https://docs.aws.amazon.com/iot/latest/developerguide/iot-rules.html), [Amazon DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Introduction.html) tables, [AWS Step Function](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html) and [AWS Lambda](https://docs.aws.amazon.com/lambda/latest/dg/welcome.html) functions will be created by [AWS CloudFormation](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/Welcome.html). There is one CloudFormation template for the slave region and one for the master region. Please refer to the README to launch the templates.

The series of Jupyter notebooks let you create an AWS IoT multi-region setup based on the resources launched by CloudFormation. An example to connect devices is also provided as well as some examples to persist data across regions. You'll find the instructions in the Jupyter notebooks.

To create a multi-region setup for AWS IoT you must bring your own CA. It must be registered in every region which applies to the multi-region setup. The CA will be created with the AWS Certificate Manager's [Private Certificate Authority](https://docs.aws.amazon.com/acm-pca/latest/userguide/PcaWelcome.html) (PCA).

AWS Certificate Manager Private Certificate Authority is a managed private CA service with which you can easily and securely manage your certificate authority infrastructure and your private certificates.

The devices certificates issued by the PCA will be automatically regiseterd using [Just-in-Time registration](https://aws.amazon.com/blogs/iot/just-in-time-registration-of-device-certificates-on-aws-iot/).

If you run the Jupyter notebooks on Amazon EC2 or an Amazon SageMaker notebook instance you need to attach the following permissions to the instance.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "acm-pca:*",
                "dynamodb:CreateGlobalTable",
                "dynamodb:DescribeGlobalTable",
                "iot:*",
                "iam:CreateRole",
                "iam:AttachRolePolicy",
                "iam:PassRole",
                "iam:GetRole",
                "lambda:AddPermission",
                "lambda:GetPolicy",
                "lambda:RemovePermission",
                "sns:*",
                "sqs:*"
            ],
            "Resource": "*"
        }
    ]
}
```


## Library

In [ ]:
from os.path import exists, join

## Shared Variables
Variables which will be used in other notebooks.

Feel free to modify the variable to you needs.

* **aws_region_pca** is the region where you set up the private CA
* **aws_region_master** is the AWS IoT master region
* **aws_region_slave** is the AWS IoT slave region

In [ ]:
CA_subject = {"C": "DE", "O": "AWS", "OU": "IoT", "ST": "Berlin", "L": "Berlin", "CN": "Multi Region CA"}
CA_directory = 'CA_{}'.format(CA_subject['CN'])
CA_key = 'ca.key.pem'
CA_cert = 'ca.crt.pem'

PCA_directory = join(CA_directory, 'PCA')

config = {}
config['aws_region_pca'] = "eu-west-1"
config['aws_region_master'] = "us-west-2"
config['aws_region_slave'] = "us-east-1"
config['CA_directory'] = CA_directory
config['CA_key'] = CA_key
config['CA_cert'] = CA_cert
config['PCA_directory'] = PCA_directory
config['CA_subject'] = CA_subject
config['Sub_CN'] = 'Subordinated IoT Device CA for multi region'
config['dynamo_provisioning_table'] = 'IoTMRProvisioning'
%store config

## Launch CloudFormation Stacks

Before you continue with the next notebook launch two CloudFormation stacks:

* ![master-region.json](../cfn/master-region.json)
* ![slave-region.json](../cfn/slave-region.json)

### Preparation
The CloudFormation stacks will create Lambda functions. You can find the code in the folder `../lambda`. The CloudFormation template requires to have the Lambda code as zip files in an S3 bucket. The Lambda function for Just-in-Time registration makes use of the pyOpenSSL library which is by default not installed in the Lambda environment. Before you create the zip files the pyOpenSSL library must be installed locally into the directory where the Lambda function is stored.

The following commands will create the required zip files. Execute the commands from within the `lambda` directory:

```
cd iot-mr-jitr
pip install pyOpenSSL -t .
zip ../iot-mr-jitr.zip -r .
cd ..

cd iot-mr-cross-region
zip ../iot-mr-cross-region.zip lambda_function.py
cd ..

cd sfn-iot-mr-dynamo-trigger
zip ../sfn-iot-mr-dynamo-trigger.zip lambda_function.py 
cd ..

cd sfn-iot-mr-thing-crud
zip ../sfn-iot-mr-thing-crud.zip lambda_function.py 
cd ..

cd sfn-iot-mr-thing-type-crud
zip ../sfn-iot-mr-thing-type-crud.zip lambda_function.py 
cd ..

```

* If you don't have a bucket in your master and slave region create one in each region
* Master region:
    * Copy the file `iot-mr-jitr.zip` to your S3 bucket
* Slave region:
    * Copy all zip files from the `lambda` folder to the S3 bucket

### Launch stacks
When launching each CloudFormation stack you need to provide the S3 bucket name where you have stored the code for the Lambda functions.

* Launch the stack **master-region.json** in the region that you have defined as master region above.

* Launch the stack **slave-region.json** in the region that you have defined as slave region above.

The CloudFormation stacks will create several AWS resources that are required to implement the IoT master-slave setup in the architecture diagram below. 

In the subsequent Jupyter notebooks you will do some hands-on exercises for an IoT multi-region setup.

## Architecture

![iot-mr-provisioning.png](iot-mr-provisioning.png)